# Lab 2: Azure AI Search RAG 지식 베이스 구성하기

## 개요 (Overview)

이 노트북에서는 Azure AI Search를 사용하여 RAG(Retrieval-Augmented Generation) 지식 베이스를 구성합니다.

### 아키텍처 (Architecture)

```
┌─────────────────────────────────────────────────────────┐
│                  RAG Pipeline                            │
│                                                          │
│  Documents (JSON) → Embeddings (OpenAI)                 │
│                         ↓                                │
│              Azure AI Search Index                       │
│           (Vector Store + Keyword Search)                │
│                         ↓                                │
│            Hybrid Search (Vector + BM25)                 │
│                         ↓                                │
│              Research Agent Query                        │
└─────────────────────────────────────────────────────────┘
```

### 학습 목표 (Learning Objectives)

이 실습을 완료하면 다음을 할 수 있게 됩니다:

1. ✅ Azure AI Search 인덱스 스키마 설계 및 생성
2. ✅ Azure OpenAI로 텍스트 임베딩 생성
3. ✅ 벡터 및 키워드 검색을 위한 문서 업로드
4. ✅ 하이브리드 검색 (벡터 + BM25) 실행 및 테스트
5. ✅ RAG 파이프라인 성능 평가 및 최적화

### 사용할 데이터 (Data)

- **data/knowledge-base.json**: 50개의 다양한 한국 여행지 정보
- **카테고리**: 자연/힐링, 문화/역사, 도시/해변, 액티비티/스포츠, 먹거리/시장
- **임베딩 모델**: text-embedding-3-large (3072차원)
- **검색 테스트**: 벡터/키워드/하이브리드 검색의 강점을 명확하게 비교할 수 있는 데이터셋

---

## ⚙️ 시작하기 전에 (Before You Start)

**Python 커널을 선택하세요:**

1. 노트북 오른쪽 상단의 **"커널 선택 (Select Kernel)"** 클릭
2. **"Python Environments..."** 선택
3. **`.venv (Python 3.x.x)`** 선택 (프로젝트 루트에 생성된 가상환경)

> 💡 **GitHub Codespaces**: Codespaces에서는 자동으로 `.venv` 환경이 생성됩니다.  
> 만약 `.venv`가 보이지 않으면 터미널에서 `python -m venv .venv`로 생성하세요.

---

## 1. 사전 요구 사항 확인 (Prerequisites Check)

다음 도구들이 설치되어 있는지 확인합니다:

- Python 3.9 이상
- Azure CLI
- Azure Developer CLI (azd)
- Docker (Container Apps 배포 시 필요)

In [ ]:
import sys, subprocess, os
import platform

# 운영체제에 따라 PATH 설정 (macOS, Linux, Codespaces 모두 지원)
system = platform.system()
if system == 'Darwin':  # macOS
    # Homebrew 경로 추가 (Intel & Apple Silicon)
    extra_paths = '/opt/homebrew/bin:/usr/local/bin'
elif system == 'Linux':  # Linux / Codespaces
    # 일반적인 Linux 바이너리 경로
    extra_paths = '/usr/local/bin:/usr/bin:/home/codespace/.local/bin'
else:  # Windows
    extra_paths = ''

if extra_paths:
    os.environ['PATH'] = extra_paths + ':' + os.environ.get('PATH', '')

def check(cmd, name):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, timeout=3, env=os.environ)
        print(f"{'✓' if result.returncode == 0 else '✗'} {name}")
    except Exception as e:
        print(f"✗ {name}")

print("=== Prerequisites Check ===")
print(f"✓ Python {sys.version.split()[0]} ({system})")
check("az --version", "Azure CLI")
check("azd version", "Azure Developer CLI")
check("docker --version", "Docker")
print("="*50)

## 2. 필수 패키지 설치 (Install Required Packages)

Azure AI 관련 필수 패키지를 설치합니다. GitHub Codespace에서 실행하는 경우 대부분의 패키지가 이미 설치되어 있을 수 있습니다.

In [ ]:
# 필수 패키지 설치
import subprocess
import sys

packages = [
    "azure-search-documents",
    "azure-identity",
    "openai",
    "python-dotenv"
]

print("=== Installing Required Packages ===\n")

for package in packages:
    print(f"Installing {package}...")
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", package],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print(f"✅ {package} installed")
    else:
        print(f"⚠️  {package} may already be installed or failed to install")

print("\n" + "="*50)
print("✅ Package installation completed!")

## 3. 설정 파일 로드 (Load Configuration)

In [ ]:
# Notebook 1에서 저장한 설정 파일 로드
import json
import os

config_path = "./config.json"

if not os.path.exists(config_path):
    raise FileNotFoundError(
        f"❌ 설정 파일을 찾을 수 없습니다: {config_path}\n"
        "먼저 Notebook 1 (01_deploy_azure_resources.ipynb)을 실행하세요."
    )

with open(config_path, 'r') as f:
    config = json.load(f)

# 필수 설정 확인 (Managed Identity 사용으로 키 불필요)
required_keys = ["search_endpoint", "project_connection_string"]
missing_keys = [key for key in required_keys if not config.get(key)]

if missing_keys:
    raise ValueError(f"❌ 필수 설정이 누락되었습니다: {', '.join(missing_keys)}")

print("✅ 설정 파일 로드 완료")
print(f"📍 Search Endpoint: {config['search_endpoint']}")
print(f"📍 AI Project Connection: {'✓ Set' if config['project_connection_string'] else '✗ Missing'}")

## 4. Azure 인증 (Azure Authentication)

Lab 1에서 이미 Azure에 로그인했지만, 세션이 만료되었을 수 있으므로 인증 상태를 확인하고 필요시 재인증합니다.

### 테넌트 ID 설정 안내

**대부분의 경우**: 테넌트 ID를 지정하지 않아도 됩니다. `tenant_id` 변수를 `"<YOUR_TENANT_ID>"` 또는 `None`으로 두고 실행하세요.

**테넌트 ID가 필요한 경우**:
- ✅ 여러 조직(회사)의 Azure 테넌트에 접근 권한이 있는 경우
- ✅ 특정 조직의 리소스로만 작업해야 하는 경우
- ✅ 로그인 시 "multiple tenants" 관련 오류가 발생하는 경우

**테넌트 ID 확인 방법**:
- Azure Portal → Azure Active Directory → 개요 → 테넌트 ID 복사
- 또는 조직 관리자에게 문의

In [ ]:
import subprocess, json

print("=== Azure Authentication ===")
print("ℹ️  인증 상태를 확인하고 필요시 로그인합니다.\n")

# 테넌트 ID를 여기에 입력하세요 (선택사항)
# 예: tenant_id = "16b3c013-d300-468d-ac64-7eda0820b6d3"
tenant_id = "<YOUR_TENANT_ID>"  # 또는 None으로 설정하면 기본 테넌트 사용

# Azure CLI 인증 상태 확인
az_account = subprocess.run("az account show", shell=True, capture_output=True, text=True)

if az_account.returncode == 0:
    account_info = json.loads(az_account.stdout)
    print(f"✅ Azure CLI 인증 완료 (기존 세션 사용)")
    print(f"   구독: {account_info.get('name', 'N/A')}")
    print(f"   테넌트: {account_info.get('tenantId', 'N/A')}")
else:
    print("⚠️  Azure CLI 인증이 필요합니다. 브라우저가 열립니다...")
    # 테넌트 ID가 설정되어 있으면 해당 테넌트로 로그인
    if tenant_id and tenant_id != "<YOUR_TENANT_ID>":
        az_login = subprocess.run(f"az login --tenant {tenant_id}", shell=True)
    else:
        az_login = subprocess.run("az login", shell=True)
    
    if az_login.returncode == 0:
        print("✅ Azure CLI 로그인 완료")
    else:
        raise Exception("❌ Azure CLI 로그인 실패")

print("="*50)

## 5. 지식 베이스 데이터 로드 (Load Knowledge Base Data)

In [ ]:
# 지식 베이스 JSON 파일 로드
knowledge_base_path = "./data/knowledge-base.json"

if not os.path.exists(knowledge_base_path):
    raise FileNotFoundError(f"❌ 지식 베이스 파일을 찾을 수 없습니다: {knowledge_base_path}")

with open(knowledge_base_path, 'r', encoding='utf-8') as f:
    knowledge_base = json.load(f)

# JSON이 직접 배열이거나 documents 래퍼가 있을 수 있음
if isinstance(knowledge_base, list):
    documents = knowledge_base
else:
    documents = knowledge_base.get("documents", [])

print(f"✅ 여행지 지식 베이스 로드 완료")
print(f"📚 총 여행지 개수: {len(documents)}")
print(f"\n📂 카테고리별 여행지 수:")

# 카테고리별 분류
from collections import Counter
categories = Counter(doc.get("section", doc.get("category", "기타")) for doc in documents)
for category, count in sorted(categories.items()):
    print(f"  • {category}: {count}개")

# 첫 번째 문서 샘플 출력
if documents:
    print(f"\n📄 샘플 여행지:")
    sample = documents[0]
    print(f"  ID: {sample['id']}")
    print(f"  제목: {sample['title']}")
    print(f"  카테고리: {sample['category']}")
    print(f"  섹션: {sample.get('section', 'N/A')}")
    print(f"  내용 길이: {len(sample['content'])} 자")
    if 'metadata' in sample and 'tags' in sample['metadata']:
        print(f"  태그: {', '.join(sample['metadata']['tags'][:5])}")

## 6. Azure AI Search 인덱스 생성 (Create Search Index)

### 📋 인덱스 스키마 설계

RAG 시스템의 핵심은 **효율적인 인덱스 스키마**입니다. 이 Lab에서 생성하는 인덱스 구조:

---

### 🔑 주요 필드 설명

| 필드 | 타입 | 역할 | 속성 |
|------|------|------|------|
| **id** | String | 고유 식별자 | `key=True` (필수) |
| **title** | String | 문서 제목 | `searchable=True` (키워드 검색) |
| **content** | String | 본문 내용 | `searchable=True`, 한국어 분석기 |
| **contentVector** | Float[] | 임베딩 벡터 | `dimensions=3072` (벡터 검색) |
| **category** | String | 카테고리 | `filterable=True` (필터링) |
| **tags** | String[] | 태그 목록 | `filterable=True` (다중 필터) |

---

### 🧠 벡터 검색 설정 (HNSW 알고리즘)

**contentVector 필드의 핵심 설정:**

```python
VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="hnsw-config",
            parameters=HnswParameters(
                m=4,                    # 그래프 연결 수
                ef_construction=400,    # 인덱싱 품질
                metric="cosine"         # 유사도 메트릭
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="vector-profile",
            algorithm_configuration_name="hnsw-config"
        )
    ]
)
```

**파라미터 의미:**

| 파라미터 | 값 | 의미 | 영향 |
|----------|-----|------|------|
| **m** | 4 | 노드당 연결 수 | 높을수록 정확하지만 느림 |
| **ef_construction** | 400 | 빌드 시 탐색 깊이 | 높을수록 인덱스 품질 향상 |
| **metric** | cosine | 유사도 계산 방식 | 임베딩에 최적화 |

**HNSW vs 기타 알고리즘:**

| 알고리즘 | 검색 속도 | 정확도 | 메모리 | 인덱스 빌드 속도 | 권장 용도 |
|---------|----------|--------|--------|-----------------|-----------|
| **HNSW** | ⚡⚡⚡ 빠름 | ⭐⭐⭐ 높음 (근사) | 중간 | 빠름 | **프로덕션 RAG** ⭐ |
| **Exhaustive KNN** | ⚡ 느림 | ⭐⭐⭐⭐ 완벽 (100%) | 낮음 | 즉시 | 소규모 (<1K 문서), 최고 정확도 필요 시 |
| Flat (Brute Force) | 느림 | 완벽 | 낮음 | 즉시 | 소규모 (<1K 문서) |
| IVF | 빠름 | 중간 | 높음 | 느림 | 대규모 (>1M 문서) |


**이 Lab 선택:** HNSW - 50개 문서에 최적이며 확장 가능

---

### 📊 인덱스 구성 요약

생성된 인덱스의 최종 구성:

```yaml
인덱스명: agentic-ai-knowledge-base
필드 수: 6개
  - 키워드 검색: title, content (ko.microsoft)
  - 벡터 검색: contentVector (3072차원, HNSW)
  - 필터링: category, tags
벡터 알고리즘: HNSW (m=4, ef_construction=400)
언어 지원: 한국어 (ko.microsoft 분석기)
```

**다음 단계:** 이 스키마에 맞춰 문서를 임베딩하고 업로드합니다! 📤

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    SimpleField,
    SearchableField
)
from azure.core.credentials import AzureKeyCredential
import subprocess

# Azure AI Search Admin Key 가져오기
print("🔑 AI Search Admin Key 가져오는 중...")
search_service_name = config.get("search_service_name", "")
resource_group = config.get("resource_group", "")

key_result = subprocess.run(
    f"az search admin-key show --resource-group {resource_group} --service-name {search_service_name} --query primaryKey -o tsv",
    shell=True,
    capture_output=True,
    text=True
)

if key_result.returncode != 0:
    raise Exception(f"❌ Admin Key를 가져올 수 없습니다: {key_result.stderr}")

search_admin_key = key_result.stdout.strip()
print("✅ Admin Key 획득 완료")

# Search Index Client 생성 (Admin Key 인증)
index_client = SearchIndexClient(
    endpoint=config["search_endpoint"],
    credential=AzureKeyCredential(search_admin_key)
)
print("✅ Search Index Client 생성 완료 (Admin Key 인증)")

# 인덱스 이름 설정
index_name = "ai-agent-knowledge-base"

# 인덱스 스키마 정의
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="title", type=SearchFieldDataType.String, 
                   filterable=True, sortable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, 
                   analyzer_name="ko.microsoft"),  # 한국어 분석기
    SimpleField(name="category", type=SearchFieldDataType.String, 
               filterable=True, sortable=True, facetable=True),
    SimpleField(name="section", type=SearchFieldDataType.String, 
               filterable=True, sortable=False),
    SearchField(
        name="contentVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=3072,  # text-embedding-3-large
        vector_search_profile_name="vector-profile"
    )
]

# 벡터 검색 설정
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(name="hnsw-algorithm")
    ],
    profiles=[
        VectorSearchProfile(
            name="vector-profile",
            algorithm_configuration_name="hnsw-algorithm"
        )
    ]
)

# 인덱스 생성
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search
)

# 기존 인덱스가 있으면 삭제 후 재생성
try:
    index_client.delete_index(index_name)
    print(f"⚠️ 기존 인덱스 '{index_name}' 삭제됨")
except:
    pass

result = index_client.create_index(index)
print(f"✅ 인덱스 생성 완료: {result.name}")
print(f"📊 필드 개수: {len(result.fields)}")
print("🔍 벡터 검색: 활성화 (3072 차원)")


## 7. Azure OpenAI로 임베딩 생성 (Generate Embeddings with Azure OpenAI)

### 🧠 텍스트 임베딩이란?

텍스트를 숫자 벡터로 변환하여 컴퓨터가 **의미**를 이해하고 비교할 수 있게 합니다.

```
"에이전트 보안"  →  [0.123, -0.456, ..., 0.234]  (3072개 숫자)
"agent security" →  [0.119, -0.451, ..., 0.228]  (매우 유사한 벡터!)
```

**핵심:** 의미가 유사한 텍스트 = 유사한 벡터 → RAG 검색의 기반

---

### 🎯 text-embedding-3-large 모델

**OpenAI의 최신 임베딩 모델 (2024년 출시)**

| 모델 | 차원 | 성능 | 적합한 용도 |
|------|------|------|------------|
| **text-embedding-3-large** | 3072 | ⭐⭐⭐⭐⭐ | **프로덕션 RAG (권장)** |
| text-embedding-3-small | 1536 | ⭐⭐⭐⭐ | 빠른 프로토타입, 비용 절감 |
| text-embedding-ada-002 | 1536 | ⭐⭐⭐ | 구 모델 (레거시) |

**3072 차원의 의미:**
- 더 많은 차원 = 더 세밀한 의미 구별 가능
- MTEB 벤치마크: **64.6%** (ada-002: 61.0%)
- 긴 텍스트 처리 가능 (~8,191 토큰)

---

### 📐 임베딩 생성 과정

```
텍스트 입력 → 토큰화 → Transformer 처리 → 벡터 생성 → 정규화
```

**이 Lab에서의 구현:**
```python
def generate_embedding(text: str) -> list[float]:
    response = openai_client.embeddings.create(
        input=text,
        model="text-embedding-3-large",
        dimensions=3072  # 명시적으로 3072 차원 지정
    )
    return response.data[0].embedding

# 제목 + 내용 결합하여 임베딩 생성
text_to_embed = f"{doc['title']}\n\n{doc['content']}"
doc["contentVector"] = generate_embedding(text_to_embed)
```

---

In [ ]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import time

# Azure OpenAI 클라이언트 생성 (Managed Identity 사용)
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

# Project connection string에서 OpenAI 엔드포인트 추출
# 형식 1: https://aoai-xxx.services.ai.azure.com/api/projects/proj-xxx;...
# 형식 2: workspace=...;subscription_id=...;resource_group=...;aiservices_name=...
import re

project_conn_str = config['project_connection_string']

# URL에서 AI Services 이름 추출 (형식 1)
url_match = re.match(r'https://([^.]+)\.', project_conn_str)
if url_match:
    aiservices_name = url_match.group(1)
    openai_endpoint = f"https://{aiservices_name}.openai.azure.com/"
else:
    # 키-값 형식에서 추출 (형식 2)
    conn_parts = {}
    for part in project_conn_str.split(';'):
        if '=' in part:
            key, value = part.split('=', 1)
            conn_parts[key] = value
    
    aiservices_name = conn_parts.get('aiservices_name', '')
    if not aiservices_name:
        raise ValueError("❌ AI Services 이름을 찾을 수 없습니다. config.json의 project_connection_string을 확인하세요.")
    
    openai_endpoint = f"https://{aiservices_name}.openai.azure.com/"

print(f"🔗 Azure OpenAI Endpoint: {openai_endpoint}")

openai_client = AzureOpenAI(
    azure_ad_token_provider=token_provider,
    api_version="2024-02-01",
    azure_endpoint=openai_endpoint
)

# 임베딩 모델 설정
embedding_model = "text-embedding-3-large"

def generate_embedding(text: str) -> list[float]:
    """텍스트를 벡터로 변환"""
    response = openai_client.embeddings.create(
        input=text,
        model=embedding_model,
        dimensions=3072  # 명시적으로 3072 차원으로 설정
    )
    return response.data[0].embedding

# 모든 문서에 대해 임베딩 생성
print("🔄 임베딩 생성 중...")
print(f"📄 처리할 문서: {len(documents)}개")

for i, doc in enumerate(documents, 1):
    # 제목과 내용을 결합하여 임베딩 생성
    text_to_embed = f"{doc['title']}\n\n{doc['content']}"
    doc["contentVector"] = generate_embedding(text_to_embed)
    
    print(f"  [{i}/{len(documents)}] {doc['title'][:50]}... ✓")
    
    # Rate limit 방지 (TPM 제한 고려)
    if i < len(documents):
        time.sleep(0.5)

print(f"\n✅ 임베딩 생성 완료")
print(f"📊 벡터 차원: {len(documents[0]['contentVector'])}차원 (3072차원)")
print(f"💾 메모리 사용량: ~{len(documents) * 3072 * 4 / 1024 / 1024:.2f} MB")


## 8. 문서를 Azure AI Search에 업로드 (Upload Documents to Azure AI Search)

### 📤 배치 업로드 전략

Azure AI Search는 **배치 업로드**를 권장합니다:

| 배치 크기 | 처리 속도 | 권장 상황 |
|-----------|-----------|-----------|
| 1-10개 | 느림 | 실시간 단일 문서 |
| **10-100개** | **빠름 ⭐** | **일반적인 인덱싱 (권장)** |
| 100-1000개 | 매우 빠름 | 대량 초기 로드 |

**이 Lab 사용: 50개 문서 → 단일 배치 (최적)**

---

### 🔧 upload_documents() 메서드

```python
search_client.upload_documents(documents=documents_with_embeddings)
```

**내부 동작:**
1. **검증**: 필수 필드 체크 (`id`, `contentVector` 등)
2. **직렬화**: JSON 배열로 변환
3. **HTTP POST**: `/docs/index` 엔드포인트
4. **응답 처리**: 성공/실패 문서별 결과 반환

**응답 예시:**
```json
{
  "value": [
    {"key": "doc1", "status": true, "statusCode": 201},
    {"key": "doc2", "status": true, "statusCode": 201}
  ]
}
```


In [ ]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Search Client 생성 (문서 업로드용)
search_client = SearchClient(
    endpoint=config["search_endpoint"],
    index_name=index_name,
    credential=AzureKeyCredential(search_admin_key)
)
print("✅ Search Client 생성 완료 (문서 업로드용)")

# 업로드할 문서 정리 (인덱스 스키마에 맞는 필드만 포함)
allowed_fields = {"id", "title", "content", "category", "section", "contentVector"}
cleaned_documents = []

for doc in documents:
    cleaned_doc = {key: value for key, value in doc.items() if key in allowed_fields}
    cleaned_documents.append(cleaned_doc)

print(f"\n📦 업로드 준비:")
print(f"   - 문서 개수: {len(cleaned_documents)}개")
print(f"   - 필드: {', '.join(allowed_fields)}")
print(f"   - 벡터 차원: {len(cleaned_documents[0]['contentVector'])}")

# 문서 업로드 (배치 단위)
print(f"\n🔄 문서 업로드 중...")

try:
    result = search_client.upload_documents(documents=cleaned_documents)
    
    # 업로드 결과 분석
    succeeded = sum(1 for r in result if r.succeeded)
    failed = len(result) - succeeded
    
    print(f"\n✅ 업로드 완료!")
    print(f"   - 성공: {succeeded}개")
    print(f"   - 실패: {failed}개")
    
    if failed > 0:
        print(f"\n⚠️ 실패한 문서:")
        for r in result:
            if not r.succeeded:
                print(f"   - {r.key}: {r.error_message}")
    
except Exception as e:
    print(f"❌ 업로드 실패: {str(e)}")
    raise

print(f"\n📊 인덱스 상태:")
print(f"   - 인덱스명: {index_name}")
print(f"   - 총 문서: {len(cleaned_documents)}개")
print(f"   - 검색 준비 완료! 🎉")


## 9. 하이브리드 검색 실습 (Hybrid Search Example)

### 🔍 하이브리드 검색이란?

**하이브리드 검색**은 **벡터 검색**과 **키워드 검색**을 결합하여 두 가지 장점을 모두 활용합니다:

| 검색 방식 | 알고리즘 | 강점 | 약점 |
|-----------|----------|------|------|
| **키워드 검색** | BM25 | 정확한 단어/구문 매칭 | 동의어·의미 이해 불가 |
| **벡터 검색** | 코사인 유사도 | 의미적 유사성 이해 | 정확한 용어 매칭 약함 |
| **하이브리드** | RRF (순위 융합) | **두 가지 장점 결합** ⭐ | 계산 비용 약간 증가 |

---

### ⚙️ RRF (Reciprocal Rank Fusion) 알고리즘

하이브리드 검색은 **RRF**를 사용하여 두 검색 결과를 융합합니다:

```
RRF Score = 1/(k + 벡터_순위) + 1/(k + 키워드_순위)
           (k = 60, Azure AI Search 기본값)
```

**예시:**
- 문서 A: 벡터 1위, 키워드 3위 → RRF = 1/61 + 1/63 = 0.0323
- 문서 B: 벡터 2위, 키워드 1위 → RRF = 1/62 + 1/61 = 0.0325 ← **높음 (우선 순위)**

---

### 💡 실습 쿼리 분석

**쿼리:** "바다가 보이는 힐링 여행지"

이 쿼리는 하이브리드 검색에 최적화되어 있습니다:
- **키워드 강점**: "바다" (정확한 장소 특징)
- **벡터 강점**: "힐링 여행지" (개념적 질문)
- **하이브리드 효과**: 정확한 위치 특징 + 분위기/느낌을 모두 고려

다음 셀에서 검색 실행 후, **섹션 10**에서 3가지 검색 방식을 비교합니다.

In [ ]:
from azure.search.documents.models import VectorizedQuery

# ✅ 실습 쿼리: 장소 특징 + 개념적 질문 (하이브리드 검색에 최적)
# 키워드("바다")와 의미("힐링")가 모두 중요함
test_query = "바다가 보이는 힐링 여행지"

print(f"🔍 검색 쿼리: '{test_query}'")
print(f"📌 이 쿼리는 정확한 장소 특징(바다)과 개념적 의미(힐링)를 모두 포함합니다.")
print(f"   → 하이브리드 검색이 가장 정확한 결과를 제공할 것으로 예상됩니다.\n")

# 1️⃣ 쿼리를 벡터로 변환 (임베딩)
print("🔄 쿼리 임베딩 생성 중...")
query_vector = generate_embedding(test_query)
print(f"✅ 임베딩 생성 완료 (차원: {len(query_vector)})\n")

# 2️⃣ 벡터 쿼리 객체 생성
vector_query = VectorizedQuery(
    vector=query_vector,         # 쿼리 임베딩 벡터
    k_nearest_neighbors=5,       # 상위 5개 유사 문서 검색
    fields="contentVector"       # 검색 대상 벡터 필드
)

# 3️⃣ 하이브리드 검색 실행 (벡터 + 키워드)
print("🔍 하이브리드 검색 실행 중...")
results = search_client.search(
    search_text=test_query,      # 키워드 검색 (BM25 알고리즘)
    vector_queries=[vector_query],  # 벡터 검색 (코사인 유사도)
    select=["title", "content", "category"],
    top=5  # 상위 5개 결과만 반환
)

# 4️⃣ 검색 결과 출력
print("=" * 100)
print("📊 하이브리드 검색 결과 (벡터 + 키워드)")
print("=" * 100)

result_count = 0
for i, result in enumerate(results, 1):
    print(f"\n🔹 결과 {i}")
    print(f"   📂 카테고리: {result['category']}")
    print(f"   📄 제목: {result['title']}")
    print(f"   📝 내용 미리보기: {result['content'][:150]}...")
    result_count = i

print("\n" + "=" * 100)
print(f"✅ 검색 완료! 총 {result_count}개 여행지 검색됨")
print("💡 다음 섹션에서 키워드/벡터/하이브리드 검색을 비교합니다.")

## 10. 검색 성능 비교 (Search Performance Comparison)

### 🔬 3가지 검색 방식 비교 실험

이번 섹션에서는 **여행지 검색 시나리오**를 통해 각 검색 방식의 차이를 명확히 확인합니다.

---

### 📊 검색 방식별 특성

| 검색 방식 | 알고리즘 | 주요 장점 | 주요 단점 | 권장 사용 케이스 |
|-----------|----------|----------|----------|-----------------|
| **키워드 검색** | BM25 | 정확한 단어 매칭, 빠른 속도 | 동의어/의미 이해 불가 | **특정 지명/고유명사 검색** ⭐, 축제명, 맛집명 |
| **벡터 검색** | 코사인 유사도 | 의미 기반, 동의어 처리, 분위기 이해 | 정확한 지명 매칭 약함 | 자연어 질문, 느낌/분위기 기반 검색 |
| **하이브리드 (RRF)** | 벡터 + BM25 융합 | **두 방식의 장점 결합** ⭐ | 계산 비용 약간 증가 | **프로덕션 RAG (권장)** |

---

### 🧪 실험 시나리오: 특정 테마 여행지 검색

**쿼리:** "서핑할 수 있는 해변"

이 쿼리는 각 검색 방식의 차이를 명확히 보여줍니다:

1. **키워드 검색 (예상: 정확한 매칭)**
   - `서핑` 문자열을 정확히 포함한 여행지만 검색
   - 양양, 부산 등 서핑 명소가 상위 랭킹될 것

2. **벡터 검색 (예상: 관련 활동 포함)**
   - "수상 레저", "해양 스포츠"와 의미적으로 유사한 여행지 검색
   - 서핑이 명시되지 않아도 해양 액티비티 관련 장소 포함 가능

3. **하이브리드 검색 (예상: 균형잡힌 결과)**
   - 정확한 서핑 명소 + 관련 해양 액티비티 장소
   - 가장 포괄적인 결과 제공

---

### 🎯 검색 방식 선택 가이드

**하이브리드 검색 (90% 케이스에 권장):**
- ✅ 일반적인 사용자 질문 답변
- ✅ 복합 쿼리 (지명 + 테마/분위기)
- ✅ 정확도가 중요한 프로덕션 환경

**키워드 전용 검색:**
- ✅ 특정 지명 검색 (예: "제주도", "경주")
- ✅ 축제명/고유명사 (예: "머드축제", "유등축제")
- ✅ 특산물/음식명 (예: "대게", "비빔밥")

**벡터 전용 검색:**
- ✅ 의미적 유사성이 중요한 경우 (느낌, 분위기)
- ✅ 자연어 질문 (예: "가족과 가기 좋은 곳")
- ✅ 개념적 질문 (예: "힐링 여행" ≈ "휴식 여행")

In [ ]:
import sys
import time

# 🧪 실험: 검색 방식별 강점을 명확히 보여주는 쿼리
# "서핑": 정확한 액티비티명 (키워드 검색 강점)
# "해변": 장소 유형 (벡터 검색으로 관련 장소 발견 가능)
# → 하이브리드는 두 가지를 모두 고려

test_query = "서핑할 수 있는 해변"

print("=" * 100)
print("🧪 검색 실험: 특정 액티비티 여행지 검색")
print("=" * 100)
print(f"📌 쿼리: '{test_query}'")
print(f"💡 예상: 키워드 검색이 가장 정확할 것 (정확한 액티비티명 매칭)")
print(f"   - 키워드 검색: '서핑' 문자열을 정확히 찾음 ⭐")
print(f"   - 벡터 검색: 의미적으로 유사한 해양 액티비티 장소도 포함")
print(f"   - 하이브리드: 두 가지를 결합하여 균형잡힌 결과 제공\n")

# 쿼리 임베딩 생성 (1회만)
print("🔄 검색 쿼리 임베딩 생성 중...")
query_vector = generate_embedding(test_query)
vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, fields="contentVector")
print("✅ 임베딩 생성 완료\n")
sys.stdout.flush()  # 출력 버퍼 강제 비우기

# === 방법 1: 키워드 전용 검색 (권장: 특정 액티비티/지명 검색) ===
print("🔍 방법 1: 키워드 전용 검색 (BM25)")
print("   → '서핑' 문자열을 정확히 매칭하는 여행지 검색")
print("-" * 100)
sys.stdout.flush()

# 검색 실행 및 결과 수집 (동기적)
keyword_search_results = search_client.search(
    search_text=test_query,
    vector_queries=None,  # 벡터 검색 비활성화
    select=["title", "category", "section"],
    top=3
)
keyword_results = []
for result in keyword_search_results:
    keyword_results.append(result)

# 결과 출력
for i, r in enumerate(keyword_results, 1):
    section = r.get('section', '기타')
    print(f"   {i}. [{section}] {r['title']}")
sys.stdout.flush()
time.sleep(0.1)  # 출력 완료 대기

# === 방법 2: 벡터 전용 검색 ===
print("\n🔍 방법 2: 벡터 전용 검색 (코사인 유사도)")
print("   → 의미적으로 유사한 여행지 검색 (해양 액티비티 중심)")
print("-" * 100)
sys.stdout.flush()

# 검색 실행 및 결과 수집 (동기적)
vector_search_results = search_client.search(
    search_text=None,  # 키워드 검색 비활성화
    vector_queries=[vector_query],
    select=["title", "category", "section"],
    top=3
)
vector_results = []
for result in vector_search_results:
    vector_results.append(result)

# 결과 출력
for i, r in enumerate(vector_results, 1):
    section = r.get('section', '기타')
    print(f"   {i}. [{section}] {r['title']}")
sys.stdout.flush()
time.sleep(0.1)  # 출력 완료 대기

# === 방법 3: 하이브리드 검색 (RRF) ===
print("\n🔍 방법 3: 하이브리드 검색 (벡터 + 키워드 융합)")
print("   → 정확한 액티비티 매칭 + 의미적 유사성을 결합")
print("-" * 100)
sys.stdout.flush()

# 검색 실행 및 결과 수집 (동기적)
hybrid_search_results = search_client.search(
    search_text=test_query,  # 키워드 검색 활성화
    vector_queries=[vector_query],
    select=["title", "category", "section"],
    top=3
)
hybrid_results = []
for result in hybrid_search_results:
    hybrid_results.append(result)

# 결과 출력
for i, r in enumerate(hybrid_results, 1):
    section = r.get('section', '기타')
    print(f"   {i}. [{section}] {r['title']}")
sys.stdout.flush()
time.sleep(0.1)  # 출력 완료 대기

print("\n" + "=" * 100)
print("📊 검색 결과 분석:")
print("=" * 100)
print("✅ 키워드 검색 (이 케이스에서 가장 정확):")
print("   - '서핑' 키워드를 정확히 포함한 여행지를 상위 랭킹")
print("   - 양양 서핑, 부산 해운대 등 서핑 명소가 주로 검색됨")
print("\n⚠️  벡터 검색:")
print("   - '해양 액티비티' 관련 개념 여행지도 포함 (의미는 유사하지만 정확한 서핑 언급 없을 수 있음)")
print("   - 해변, 바다 관련 여행지가 광범위하게 검색될 수 있음")
print("\n⭐ 하이브리드 검색 (권장):")
print("   - 키워드 검색의 정확성 + 벡터 검색의 의미적 이해를 결합")
print("   - 서핑이 명시된 여행지 우선 + 관련 해양 액티비티 장소도 적절히 포함")
print("\n💡 결론: 특정 액티비티/지명 검색 시 키워드 검색이 강하지만,")
print("         실제 프로덕션에서는 하이브리드가 더 포괄적인 결과 제공")

### 📈 검색 결과 분석 가이드

위 실험 결과는 Azure AI Search의 3가지 검색 방식의 특징을 보여줍니다:

---

#### 🔑 키워드 검색 (BM25)
- **강점**: 정확한 키워드 매칭 (서핑, 한옥마을, 설악산 등)
- **약점**: 동의어/유사 개념 검색 약함 (예: "힐링" 검색 시 "휴식", "명상" 언급 문서 못 찾음)
- **권장 사용 케이스**: 
  - 특정 지명/액티비티 검색 (예: "경복궁", "제주도", "서핑")
  - 전문 용어 검색 (기술 문서 RAG인 경우)

#### 🧠 벡터 검색 (의미적 검색)
- **강점**: 의미적 유사성 검색 (예: "힐링 여행지" → 명상/요가/자연휴양 관련 장소 검색)
- **약점**: 정확한 키워드 매칭이 중요한 경우 정확도 떨어질 수 있음
- **권장 사용 케이스**:
  - 추상적인 개념 검색 (예: "가족과 함께하는 여행", "힐링이 되는 곳")
  - 다국어/동의어 검색 (임베딩 모델이 의미를 이해)

#### ⚖️ 하이브리드 검색 (RRF)
- **강점**: 키워드 + 벡터 검색 결합 → 정확성과 의미적 이해 모두 확보
- **프로덕션 권장**: 대부분의 실제 서비스에서는 하이브리드가 최적
- **작동 원리**: 
  - 키워드 검색 Top-K와 벡터 검색 Top-K를 RRF(Reciprocal Rank Fusion)로 재정렬
  - 두 방식에서 모두 상위에 랭크된 문서가 최종적으로 높은 점수 획득

---

### 🎯 실습 팁
다양한 쿼리로 실험해보세요:
```python
# 정확한 키워드 매칭이 중요한 경우
test_query = "제주도 섭지코지"

# 의미적 검색이 유리한 경우
test_query = "가족과 함께 힐링할 수 있는 자연 휴양지"

# 액티비티 중심 검색
test_query = "다이빙 스쿠버 즐길 수 있는 곳"
```

## 11. 설정 파일 업데이트 (Update Configuration)

인덱스 이름을 `config.json`에 저장하여 Notebook 3 (에이전트 배포)에서 사용할 수 있도록 합니다.

In [ ]:
# config.json 다시 로드
with open("./config.json", "r", encoding="utf-8") as f:
    config = json.load(f)

# 인덱스 이름 추가
config["search_index"] = index_name

# 업데이트된 설정 저장
with open("./config.json", "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print("✅ 설정 파일 업데이트 완료!")
print(f"   - 인덱스 이름: {index_name}")
print(f"   - 저장 위치: config.json")

## 📍 다음 단계

RAG 지식 베이스 구성을 완료하셨습니다! 이제 다음 노트북으로 순서대로 진행하세요:

1. **노트북 03**: Foundry Agent 배포 (`03_deploy_foundry_agent.ipynb`)
2. **노트북 04**: MAF 기반 Agent 배포 (`04_deploy_foundry_agent_with_maf.ipynb`)
3. **노트북 05**: MAF 워크플로우 패턴 실습 (`05_maf_workflow_patterns.ipynb`)
4. **노트북 06**: MAF Dev UI 실습 (`06_maf_dev_ui.ipynb`)
5. **노트북 07**: 에이전트 평가 (`07_evaluate_agents.ipynb`)